# Step 4. Flower Classifier - Convolutional Neural Network

This approach is better suited than pure Artificial Neural Network (ANN) because it is able to reliably capture important features in a 2D image.

Optimizer: `tf.keras.optimizers.RMSprop`

According to [Tensorflow API docs](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/experimental/RMSprop), The gist of RMSprop is to:

- Maintain a moving (discounted) average of the square of gradients
- Divide the gradient by the root of this average

Thus, this is a good choice for non-stationery problems. From observing our baseline ANN model that uses Adam optimizer and its prediction on the test dataset, we saw that the baseline model struggles to capture our non-stationary data: there is significant variance in lighting, angle, zoom level, etc. As such, RMSprop is a better choice than Adam.

Since our data is balanced, the metric we are using to evaluate its performance is **Accuracy** ($Accuracy = \frac{correct \; classifications}{all \; classifications}$), more specifically its [Tensorflow's CategoricalAccuracy Implementation](https://www.tensorflow.org/api_docs/python/tf/keras/metrics/CategoricalAccuracy).

Refer to the code below to see how it is set up, trained, and evaluated.

In [ ]:
# Imports and environmental setups

import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import RMSprop


# Custom utils functions
from utils import dataset
from utils.visualization import plot_learning_curve
from utils.visualization import visualize_16predictions
from utils.glob import TARGET_IMG_SIZE


# Reproducible results
np.random.seed(42)
tf.random.set_seed(42)

# Misc.
plt.style.use('ggplot')
model_name = 'clf-cnn'
dir_raw_dataset = '../data/raw/'
dir_train_dataset = '../data/train/'
dir_test_dataset = '../data/test/'
dir_log = '../log/' + model_name
dir_models = '../models/'
path_model = dir_models + model_name

## 3.1. Load Dataset

In [ ]:
df = dataset.load(dir_train_dataset)

In [ ]:
df.info()

In [ ]:
df.sample(n=42, random_state=42)

In [ ]:
train, val = train_test_split(df, shuffle=True, test_size=0.25, random_state=42)  # 80/20/20 - train/val/test

print(f'Train data: {train.shape[0]} samples, Validation Data: {val.shape[0]} samples.')

## 3.2. Model Setup and Training

### 3.2.1. Hyper-parameters

In [ ]:
# Model Parameters
BATCH_SIZE = 512
INPUT_DIM = (TARGET_IMG_SIZE, TARGET_IMG_SIZE, 3)  # RGB - 3 channels images
OUTPUT_CLASSES = 8  # One-hot encoded: 8 different classes

# Training Parameters
EPOCHS = 32
LEARNING_RATE = 1e-6

### 3.2.2. Model Architecture

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (4, 4), activation='relu', input_shape=INPUT_DIM),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (4, 4), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (4, 4), activation='relu'),
    tf.keras.layers.GlobalMaxPool2D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='sigmoid'),
    tf.keras.layers.Dense(OUTPUT_CLASSES),
], name=model_name)

model.compile(
    optimizer=RMSprop(learning_rate=LEARNING_RATE),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=['categorical_accuracy'],
)

In [ ]:
model.summary()

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True, to_file=f'images/{model_name}-Diagram.png')

### 3.2.3. Training

In [ ]:
train['Class'] = train['Class'].astype('str')
train_datagen = ImageDataGenerator(data_format='channels_last')
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train,
    directory=dir_train_dataset,
    x_col='ImgPath',
    y_col='Class',
    target_size=(TARGET_IMG_SIZE, TARGET_IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    seed=42
)

val['Class'] = val['Class'].astype('str')
val_datagen = ImageDataGenerator(data_format='channels_last')
val_generator = val_datagen.flow_from_dataframe(
    dataframe=val,
    directory=dir_train_dataset,
    x_col='ImgPath',
    y_col='Class',
    target_size=(TARGET_IMG_SIZE, TARGET_IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    seed=42
)

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=dir_log, write_graph=False)
history = model.fit(
    train_generator, validation_data=val_generator,
    epochs=EPOCHS,
    verbose=2, callbacks=[tensorboard_callback],
)

In [ ]:
plot_learning_curve(
    history.history['loss'],
    history.history['val_loss'],
    history.history['categorical_accuracy'],
    history.history['val_categorical_accuracy'],
    to_file=f'images/{model_name}-LearningCurve.png'
)

## 3.3. Model Performance

Evaluated against test dataset in `./data/test/`

In [ ]:
test = dataset.load(dir_test_dataset)
test['Class'] = test['Class'].astype('str')
test_datagen = ImageDataGenerator(data_format='channels_last')
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test,
    directory=dir_test_dataset,
    x_col='ImgPath',
    y_col='Class',
    target_size=(TARGET_IMG_SIZE, TARGET_IMG_SIZE),
    batch_size=1,
    class_mode='categorical',
    seed=42
)

In [ ]:
visualize_16predictions(model, test_generator, to_file=f'images/{model_name}-16SamplePredictions.png')

In [ ]:
model.evaluate(test_generator)

**☞ Observation**

Our baseline classification model seems to have reached peak capacity at around $0.5$ accuracy, higher than random chance of $\frac{1}{8}=0.125$.

From inspecting the test predictions, we can see that the noise (irrelevant details, different flower types in the same image, background, people, etc.) in our data has a negative effect on our accuracy.

This is indicative that our model can capture some, but not all important features in the training dataset, and needs further tweaking in order to achieve higher accuracy and more resistance to noise.

## 3.4. Model Exporting

In [ ]:
model.save(path_model, overwrite = True)